Function: Compute triangular discriminator to classify unfolding performance. See Table 1: https://arxiv.org/pdf/1911.09107

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import copy
from tabulate import tabulate

import energyflow as ef # for data
import omnifold

# Local imports for plotting (source: https://github.com/ericmetodiev/OmniFold/blob/master/modplot.py)
import modplot
from hist_utils import obs, calc_obs, hist_style, gen_style, truth_style, omnifold_style, multifold_style, ibu_style

2025-04-02 14:54:29.969846: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-02 14:54:29.969880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-02 14:54:29.971641: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-02 14:54:29.980795: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-02 14:54:31.446551: W tensorflow/compiler/tf2

Horovod not found, will continue with single only GPUs.


/global/homes/a/aelabd/omnifold_hvod/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
plt.rcParams['figure.figsize'] = (4,4)
plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.family'] = 'serif'

# IBU hyperparameters
N_ITER_IBU = 3

# OmniFold hyperparameters
N_ITER_OF = 5
LR_OF = 5e-5
LR_PATIENCE_OF = 10
BATCH_SIZE_OF = 1000
EPOCHS_OF = 100
NUM_HEAD_OF = 8
NUM_TRANSFORMER_OF = 4
PROJECTION_DIM_OF = 97

# Multifold hyperparameters
LAYER_SIZES = [64, 128, 64]
N_ITER_MF = 3
BATCH_SIZE_MF = 500
EPOCHS_MF = 2

# Other globals 
N_DATA = int(1.6e6)
OBS_MULTIFOLD = ['Mass', 'Mult', 'Width',  'SDMass', 'Tau21', 'zg']
N_JOBS = 16

np.random.seed(0)
import tensorflow as tf
tf.random.set_seed(0)

l10data = int(np.log10(N_DATA))
print(f"N_DATA = {N_DATA/(10**(l10data))}e{int(l10data)}")

N_DATA = 1.6e6


# 1. Load data

Pythia as the Monte Carlo data

In [3]:
data_mc_dict = ef.zjets_delphes.load('Pythia26', exclude_keys='particles', num_data=N_DATA, pad=True, source='zenodo', which='all')
obs_mc_gen = {k.replace("gen_", ""): data_mc_dict[k] for k in data_mc_dict.keys() if (('gen_' in k) and ('particle' not in k))}
obs_mc_sim = {k.replace("sim_", ""): data_mc_dict[k] for k in data_mc_dict.keys() if (('sim_' in k) and ('particle' not in k))}

obs_mc_gen['Mass'] = obs_mc_gen['jets'][:,3]
obs_mc_gen.pop('jets')
obs_mc_gen['Mult'] = obs_mc_gen.pop('mults')
obs_mc_gen['Width'] = obs_mc_gen.pop('widths')
obs_mc_gen['Tau21'] = obs_mc_gen.pop('tau2s')
obs_mc_gen['zg'] = obs_mc_gen.pop('zgs')
obs_mc_gen['SDMass'] = obs_mc_gen.pop('sdms')
obs_mc_gen = np.concatenate([obs_mc_gen[k].reshape((N_DATA,1)) for k in OBS_MULTIFOLD], axis=1)


obs_mc_sim['Mass'] = obs_mc_sim['jets'][:,3]
obs_mc_sim.pop('jets')
obs_mc_sim['Mult'] = obs_mc_sim.pop('mults')
obs_mc_sim['Width'] = obs_mc_sim.pop('widths')
obs_mc_sim['Tau21'] = obs_mc_sim.pop('tau2s')
obs_mc_sim['zg'] = obs_mc_sim.pop('zgs')
obs_mc_sim['SDMass'] = obs_mc_sim.pop('sdms')
obs_mc_sim = np.concatenate([obs_mc_sim[k].reshape((N_DATA,1)) for k in OBS_MULTIFOLD], axis=1)
obs_mc = omnifold.DataLoader(reco=copy.deepcopy(obs_mc_sim), gen=copy.deepcopy(obs_mc_gen))

INFO: Creating weights ...
INFO: Creating pass reco flag ...
INFO: Creating pass gen flag ...


Hedwig as the "true" data

In [4]:
data_nature_dict = ef.zjets_delphes.load('Herwig', exclude_keys='particles', num_data=N_DATA, pad=True, source='zenodo', which='all')
obs_nature_gen = {k.replace("gen_", ""): data_nature_dict[k] for k in data_nature_dict.keys() if (('gen_' in k) and ('particle' not in k))}
obs_nature_sim = {k.replace("sim_", ""): data_nature_dict[k] for k in data_nature_dict.keys() if (('sim_' in k) and ('particle' not in k))}

obs_nature_gen['Mass'] = obs_nature_gen['jets'][:,3]
obs_nature_gen.pop('jets')
obs_nature_gen['Mult'] = obs_nature_gen.pop('mults')
obs_nature_gen['Width'] = obs_nature_gen.pop('widths')
obs_nature_gen['Tau21'] = obs_nature_gen.pop('tau2s')
obs_nature_gen['zg'] = obs_nature_gen.pop('zgs')
obs_nature_gen['SDMass'] = obs_nature_gen.pop('sdms')
obs_nature_gen = np.concatenate([obs_nature_gen[k].reshape((N_DATA,1)) for k in OBS_MULTIFOLD], axis=1)

obs_nature_sim['Mass'] = obs_nature_sim['jets'][:,3]
obs_nature_sim.pop('jets')
obs_nature_sim['Mult'] = obs_nature_sim.pop('mults')
obs_nature_sim['Width'] = obs_nature_sim.pop('widths')
obs_nature_sim['Tau21'] = obs_nature_sim.pop('tau2s')
obs_nature_sim['zg'] = obs_nature_sim.pop('zgs')
obs_nature_sim['SDMass'] = obs_nature_sim.pop('sdms')
obs_nature_sim = np.concatenate([obs_nature_sim[k].reshape((N_DATA,1)) for k in OBS_MULTIFOLD], axis=1)

obs_nature = omnifold.DataLoader(reco=copy.deepcopy(obs_nature_sim), gen=copy.deepcopy(obs_nature_gen))


INFO: Creating weights ...
INFO: Creating pass reco flag ...
INFO: Creating pass gen flag ...


Compute IBU rq

In [5]:
print("\n\nCalculating IBU...\n")
calc_obs(obs_dict=obs, data_synth=data_mc_dict, data_real=data_nature_dict, itnum=N_ITER_IBU)



Calculating IBU...

Done with Mass
Done with Mult
Done with Width
Done with Tau21
Done with zg
Done with SDMass


# 2. Compute the "true" probability distribution, q

In [6]:
q_dict = {}
for k in OBS_MULTIFOLD:
    q_dict[k] = obs[k]["truth_hist"].copy()/sum(obs[k]["truth_hist"])
    print(q_dict[k].shape)

(50,)
(80,)
(50,)
(50,)
(50,)
(50,)


# 3. Compute the IBU probability distribution, $p_{\text{IBU}}$

In [7]:
p_ibu_dict = {}
for k in OBS_MULTIFOLD:
    p_ibu_dict[k] = obs[k]["ibu_phis"][-1].copy()/sum(obs[k]["ibu_phis"][-1])
    print(p_ibu_dict[k].shape)

(50,)
(80,)
(50,)
(50,)
(50,)
(50,)


# 4. Compute the MultiFold probability distribution, $p_{\text{MF}}$

In [8]:
# weights_mf = np.load("hist_weights/"+ f"MultiFold_niter{N_ITER_MF}_bs{BATCH_SIZE_MF}_ep{EPOCHS_MF}_layers{LAYER_SIZES[0]}_{LAYER_SIZES[1]}_{LAYER_SIZES[2]}" + "_final_weights.npy")
weights_mf = np.load(f"hist_weights_old/MultiFold_N1.6e6_niter{N_ITER_MF}_bs{BATCH_SIZE_MF}_ep{EPOCHS_MF}_weights_push.npy")

p_mf_dict = {}
for k in OBS_MULTIFOLD:
    mf_histgen, mf_histgen_unc = modplot.calc_hist(obs[k]['genobs'], weights=weights_mf, 
                                                    bins=obs[k]['bins_mc'], density=True)[:2]
    p_mf_dict[k] = mf_histgen.copy()/sum(mf_histgen)
    print(p_mf_dict[k].shape)
    

(50,)
(80,)
(50,)
(50,)
(50,)
(50,)


# 5. Compute the OmniFold probability distribution, $p_{\text{OF}}$

In [9]:
# weights_of = np.load("hist_weights/"+ f"OmniFold_niter{N_ITER_OF}_lr{LR_OF}_lrp{LR_PATIENCE_OF}_bs{BATCH_SIZE_OF}_ep{EPOCHS_OF}_nhead{NUM_HEAD_OF}_ntl{NUM_TRANSFORMER_OF}_pdim{PROJECTION_DIM_OF}" + "_final_weights.npy")
weights_of = np.load(f"hist_weights_old/OmniFold_N1.6e6_niter{N_ITER_OF}_bs{BATCH_SIZE_OF}_ep{EPOCHS_OF}_final_weights.npy")

p_of_dict = {}
for k in OBS_MULTIFOLD:
    of_histgen, of_histgen_unc = modplot.calc_hist(obs[k]['genobs'], weights=weights_of, 
                                                    bins=obs[k]['bins_mc'], density=True)[:2]
    p_of_dict[k] = of_histgen.copy()/sum(of_histgen)
    print(p_of_dict[k].shape)
    

(50,)
(80,)
(50,)
(50,)
(50,)
(50,)


# 6. Compute triangular discriminant table

$ \Delta(p, q) = \frac{1}{2} \int \frac{(p(\lambda)-q(\lambda))^2}{p(\lambda)+q(\lambda)} d\lambda$

In [10]:
def compute_delta(p, q):
    numerator = (p-q)**2
    denominator = p+q+1e-50
    delta = 1e3*(1/2)*np.sum(numerator/denominator)
    return delta

delta_dict = {"IBU": {}, "MultiFold": {}, "OmniFold": {}}
for k in OBS_MULTIFOLD:
    delta_dict['IBU'][k] = compute_delta(p_ibu_dict[k], q_dict[k])
    delta_dict['MultiFold'][k] = compute_delta(p_mf_dict[k], q_dict[k])
    delta_dict['OmniFold'][k] = compute_delta(p_of_dict[k], q_dict[k])

In [11]:
headers = ["Method", "m", "M", "w", "SDMass", "t21", "zg"]
tri_disc_data = [
    ["OmniFold"],
    ["MultiFold"],
    ["IBU"],
]
tri_disc_data[0].extend(delta_dict["OmniFold"].values())
tri_disc_data[1].extend(delta_dict["MultiFold"].values())
tri_disc_data[2].extend(delta_dict["IBU"].values())


print(tabulate(tri_disc_data, headers=headers, tablefmt="grid"))

+-----------+----------+----------+-----------+----------+---------+----------+
| Method    |        m |        M |         w |   SDMass |     t21 |       zg |
+===========+==========+==========+===========+==========+=========+==========+
| OmniFold  |  2.80544 | 0.445316 | 0.760862  | 1.25528  | 4.59479 | 0.90784  |
+-----------+----------+----------+-----------+----------+---------+----------+
| MultiFold | 11.3429  | 4.78266  | 2.45608   | 3.89037  | 2.45785 | 0.810493 |
+-----------+----------+----------+-----------+----------+---------+----------+
| IBU       |  7.19566 | 1.11647  | 0.0713979 | 0.439882 | 3.53143 | 0.976731 |
+-----------+----------+----------+-----------+----------+---------+----------+
